In [2]:
"""{¶ 2} On May 30, 1998, plaintiffs-appellees Horace and Terry
Daniel (appellees) purchased a used 1993 Nissan Maxima from Motorcars, a now dissolved corporation whose former place of business was 18122 Rockside Road, Bedford, Ohio. The sales contract, as well as the odometer disclosure, that both appellees and a Motorcars’ representative signed listed the vehicle’s mileage
−3−
as 75,928. Appellees paid $12,137.75 for the vehicle. On June 9, 1998, the state of Ohio issued appellees a certificate of title for the vehicle. On this certificate, the mileage is listed as 75,928; however, the certificate indicates that the odometer status is “exceeded.”1 According to the record, when a vehicle’s mileage has exceeded its mechanical limits, the vehicle has been driven in excess of 100,000 miles. Additionally, a 1993 Nissan Maxima’s odometer restarts at zero after it passes the 99,999 mark. Although appellees did not realize it, the vehicle’s mileage was
actually 175,928 when they purchased it."""

'{¶ 2} On May 30, 1998, plaintiffs-appellees Horace and Terry\nDaniel (appellees) purchased a used 1993 Nissan Maxima from Motorcars, a now dissolved corporation whose former place of business was 18122 Rockside Road, Bedford, Ohio. The sales contract, as well as the odometer disclosure, that both appellees and a Motorcars’ representative signed listed the vehicle’s mileage\n−3−\nas 75,928. Appellees paid $12,137.75 for the vehicle. On June 9, 1998, the state of Ohio issued appellees a certificate of title for the vehicle. On this certificate, the mileage is listed as 75,928; however, the certificate indicates that the odometer status is “exceeded.”1 According to the record, when a vehicle’s mileage has exceeded its mechanical limits, the vehicle has been driven in excess of 100,000 miles. Additionally, a 1993 Nissan Maxima’s odometer restarts at zero after it passes the 99,999 mark. Although appellees did not realize it, the vehicle’s mileage was\nactually 175,928 when they purchased 

In [3]:
from openai import OpenAI
import pandas as pd
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    from tqdm.autonotebook import tqdm
from tqdm.auto import tqdm  # this is our progress bar
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from dotenv import load_dotenv
import os
load_dotenv()
# Set up the Pinecone vector database
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
client = OpenAI()
model_name = "text-embedding-ada-002"
embed = OpenAIEmbeddings(model=model_name, openai_api_key=OPENAI_API_KEY)
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
PINECONE_ENV = "us-east-1-aws"
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_ENV  # next to api key in console
)
index_name = "doc-prod-test"
index = pinecone.Index(index_name)


QUERY


In [9]:

query = "Some people bought a car from a dealer and did not know that the mileage on the car was wrong."# Create an embedding for the query
res = client.embeddings.create(input=[query],
model=model_name)

if res and res.data:
    embeddings = res.data[0].embedding
    
results = index.query(
    vector=embeddings,
    top_k=10,
    include_metadata=True
)

In [11]:
results


{'matches': [{'id': 'fe434d61-e707-47fc-b695-8a418026954a',
              'metadata': {'case_no': '2007-Ohio-6406',
                           'district': '11',
                           'page': 1.0,
                           'text': '2for $10,350.  Ms. Toy purchased the '
                                   'vehicle “‘AS IS’ - without guarantee '
                                   'express or  implied ***” and without '
                                   'knowing the actual mileage of the '
                                   'vehicle.  Although Ms. Toy  signed an '
                                   '“Odometer Disclosure Statement” '
                                   'acknowledging that the mileage on the '
                                   'odometer (41,466) was not the actual '
                                   'mileage, she believed, based upon alleged '
                                   'representations by the owner, Mr. Mazza, '
                                   'that the v

In [13]:
# Extract the relevant information from the 'similar_docs' dictionary
metadata = results['matches'][0]['metadata']
case_no = metadata['case_no']
district = metadata['district']
url = metadata['url']
text = metadata['text']
score = results['matches'][0]['score']
# doc_ids = [doc['id'] for doc in results['results']]
# scores = [doc['score'] for doc in results['results']]
# metadata = [doc['metadata'] for doc in results['results']]

# # Create a dataframe from the extracted information
# df = pd.DataFrame({
#     'doc_id': doc_ids,
#     'score': scores,
#     'metadata': metadata
# })

In [7]:
# print(type(similar_docs))
import pandas as pd
df = pd.DataFrame(similar_docs)
df.head()

ValueError: DataFrame constructor not properly called!

HYPERLINK FUNCTION


In [74]:
from docx import Document
import docx.opc.constants
from docx.oxml.shared import OxmlElement

def add_hyperlink(paragraph, text, url):
    """
    Add a hyperlink to a paragraph.
    :param paragraph: The paragraph to which the hyperlink will be added.
    :param text: The display text of the hyperlink.
    :param url: The URL the hyperlink points to.
    """
    part = paragraph.part
    r_id = part.relate_to(url, docx.opc.constants.RELATIONSHIP_TYPE.HYPERLINK, is_external=True)

    hyperlink = OxmlElement('w:hyperlink')
    hyperlink.set(docx.oxml.shared.qn('r:id'), r_id,)

    new_run = OxmlElement('w:r')
    rPr = OxmlElement('w:rPr')

    # Create text element
    text_element = OxmlElement('w:t')
    text_element.text = text
    new_run.append(rPr)
    new_run.append(text_element)

    hyperlink.append(new_run)
    paragraph._p.append(hyperlink)

    return hyperlink

MAKE WORD DOCUMENT


In [75]:
document = Document()

def doc_maker(case_no, district, url, score, text):
    document.add_heading(f'Case Number {case_no} - {score}% similarity')
    document.add_paragraph("_____________________________________________")
    document.add_paragraph(text)
    document.add_paragraph("_____________________________________________")
    para = document.add_paragraph('Download the opinion here: ')
    add_hyperlink(para, url, url)
    document.add_paragraph("")

In [77]:
def data_formatter(content):
    for doc in content:
        metadata = doc['metadata']
        score = doc['score']
        score = float("{:.2f}".format(score))*100
        case_no = metadata['case_no']
        district = metadata['district']
        text = metadata['text']
        url = metadata['url']
        year = metadata['year']
        doc_maker(case_no, district, url, score, text)
    document.save('results3.docx')
        
data_formatter(similar_docs['matches'])

In [22]:
metadata = similar_docs['matches'][0]['metadata']
score = similar_docs['matches'][0]['score']
#convert score to int and two decimal places and then to percentage
score = float("{:.2f}".format(score))*100
case_no = metadata['case_no']
district = metadata['district']
text = metadata['text']
url = metadata['url']
year = metadata['year']
# print(metadata)
print(f"case_no: {case_no}")
print(f"district: {district}")
# print(f"text is {text}")
print(f"url: {url}")
print(f"year: {str(int(year))}")
print(f"similarity score: {score}%")
print(metadata['text'])

case_no: 2014-Ohio-130
district: 2
url: https://www.supremecourt.ohio.gov/rod/docs/pdf/2/2014/2014-Ohio-130.pdf
year: 2014
similarity score: 81.0%
33 {¶ 67}   There are no extraordinary circumstances in the case before us.  The fraudulent  conduct alleged in the complaint is wrongful concealment of fi nancial misappropr iation of Mt.  Carmel’s funds.  Again, in essence, this is a cl aim of misconduct on the part  of the pastor.  If  the members of the church are aggrieved by the pastor’s conduct, the constitution provides a  remedy.  The chairman of deacons or any memb er of the congregation in good standing may  initiate charges against the pastor.  A 90-day period then ensues for resolution of the grievance.  If the charge is not resolved, the chairman or any member may call for a meeting of the congregation to consider th e pastor’s termination.  See Article VIII, Section 2 of the Mt. Carmel  Constitution.  There is no indication that S mith attempted to follow this process.   {¶ 

In [16]:
similar_docs


{'matches': [{'id': 'c684d223-5533-4314-a7a6-a87958eab495',
              'metadata': {'case_no': '2014-Ohio-130',
                           'district': '2',
                           'page': 32.0,
                           'text': '\uf02d33\uf02d {¶ 67}   There are no '
                                   'extraordinary circumstances in the case '
                                   'before us.  The fraudulent  conduct '
                                   'alleged in the complaint is wrongful '
                                   'concealment of fi nancial misappropr '
                                   'iation of Mt.  Carmel’s funds.  Again, in '
                                   'essence, this is a cl aim of misconduct on '
                                   'the part  of the pastor.  If  the members '
                                   'of the church are aggrieved by the '
                                   'pastor’s conduct, the constitution '
                                   '

SEARCHING META DATA ONLY


In [89]:
query = {"filter": {"district": {"$eq": "1"}}}
# results = index.query(query, top_k=1)


result = index.query(query, top_k=10, include_metadata=True)

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain', 'content-length': '68', 'date': 'Thu, 21 Dec 2023 15:52:12 GMT', 'server': 'envoy', 'connection': 'close'})
HTTP response body: : invalid value Starting an object on a scalar field for type vector
